In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wrangle
from sklearn.model_selection import train_test_split
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_curve, roc_auc_score, precision_recall_curve
sns.set()
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [15]:
df = pd.read_csv('data/training_v2.csv')

In [199]:
df['gcs'] = df.gcs_eyes_apache + df.gcs_motor_apache + df.gcs_verbal_apache

In [17]:
df.shape

(91713, 187)

In [18]:
df.head(3)

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,gcs
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0.0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0.0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,13.0
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,7

In [54]:
df.icu_type.value_counts()

Med-Surg ICU    46916
MICU             7343
Neuro ICU        7055
CCU-CTICU        6663
SICU             4967
CSICU            4428
Cardiac ICU      4380
CTICU            3686
Name: icu_type, dtype: int64

In [40]:
min_max_cols = []
for c in df.columns:
    if '_min' in c and c.replace('min', 'max') in df.columns:
        min_max_cols.append(c)
print(f'min_max_cols: {min_max_cols} ({len(min_max_cols)})')

min_max_cols: ['d1_diasbp_invasive_min', 'd1_diasbp_min', 'd1_diasbp_noninvasive_min', 'd1_heartrate_min', 'd1_mbp_invasive_min', 'd1_mbp_min', 'd1_mbp_noninvasive_min', 'd1_resprate_min', 'd1_spo2_min', 'd1_sysbp_invasive_min', 'd1_sysbp_min', 'd1_sysbp_noninvasive_min', 'd1_temp_min', 'h1_diasbp_invasive_min', 'h1_diasbp_min', 'h1_diasbp_noninvasive_min', 'h1_heartrate_min', 'h1_mbp_invasive_min', 'h1_mbp_min', 'h1_mbp_noninvasive_min', 'h1_resprate_min', 'h1_spo2_min', 'h1_sysbp_invasive_min', 'h1_sysbp_min', 'h1_sysbp_noninvasive_min', 'h1_temp_min', 'd1_bilirubin_min', 'd1_bun_min', 'd1_calcium_min', 'd1_creatinine_min', 'd1_glucose_min', 'd1_hco3_min', 'd1_hemaglobin_min', 'd1_hematocrit_min', 'd1_inr_min', 'd1_lactate_min', 'd1_platelets_min', 'd1_potassium_min', 'd1_sodium_min', 'd1_wbc_min', 'h1_bilirubin_min', 'h1_bun_min', 'h1_calcium_min', 'h1_creatinine_min', 'h1_glucose_min', 'h1_hco3_min', 'h1_hemaglobin_min', 'h1_hematocrit_min', 'h1_inr_min', 'h1_lactate_min', 'h1_plat

In [32]:
df['patient_id'].isnull().mean()

0.0

In [33]:

import numpy as np
import pandas as pd


from sklearn.metrics import log_loss, roc_auc_score, mean_squared_error
from sklearn.model_selection import KFold

from sklearn.preprocessing import LabelEncoder


df = pd.read_csv('data/training_v2.csv')
id_col = 'encounter_id'

df1 = pd.read_csv('data/WiDS Datathon 2020 Dictionary.csv')
df1 = df1[(df1['Data Type'] == 'string') | (df1['Data Type'] == 'binary')]
cat_features = list(df1['Variable Name'].values)
for c in cat_features:
    if c not in df1.columns or c == 'hospital_death':
        cat_features.remove(c)
print(f'cat_features: {cat_features}')

cat_features: ['bmi', 'ethnicity', 'hospital_admit_source', 'icu_admit_type', 'icu_type', 'apache_2_diagnosis', 'apache_post_operative', 'gcs_unable_apache', 'ventilated_apache', 'cirrhosis', 'hepatic_failure', 'leukemia', 'solid_tumor_with_metastasis', 'apache_2_bodysystem']


In [4]:
df1.head(2)

,Category,Variable Name,Unit of Measure,Data Type,Description,Example
3,demographic,hospital_death,None,binary,Whether the patient died during this hospitali...,0
5,demographic,bmi,kilograms/metres^2,string,The body mass index of the person on unit admi...,21.5


In [107]:
# target_cols = []
# for c in df.columns:
#     if c != id_col and c != 'hospital_death' and df[c].isnull().mean() > 0 and c not in cat_features:
#         target_cols.append({'fname': c, 'type': 'regression'})

# target_cols

In [42]:
def min_max_cols(df):
    min_max = []
    for col in df.columns:
        if '_min' in col and col.replace('_min', '_max') in df.columns:
            min_max.append(col)
    return min_max
        
def fix_min_max(df):
    min_max = min_max_cols(df)
    for col in min_max:
        vals = df[[col, col.replace('_min', '_max')]].values.copy()

        df[col] = np.nanmin(vals, axis=1)
        df[col.replace('_min', '_max')] = np.nanmax(vals, axis=1)
        
fix_min_max(df)

In [43]:
def fix_min_max(df, min_max):
    for col in min_max:
        vals = df[[col, col.replace('_min', '_max')]].values.copy()
        print(vals)
        print(col)
        df[col] = np.nanmin(vals, axis=1)
        df[col.replace('_min', '_max')] = np.nanmax(vals, axis=1)

In [5]:
df =wrangle.get_training_data()

---Acquiring the Data---
rows: 91713, columns: 186


---Handling Missing Values---
Filling nulls with mode for the following features:
	 * age
	 * bmi
	 * ethnicity
Handling nulls within rows
	 * Dropping patients with no recorded gender
		 - Number of patients dropped: 25
Filling nulls with 0 (aka False) for the following columns
	 * arf_apache
	 * intubated_apache
	 * cirrhosis
	 * aids
	 * diabetes_mellitus
	 * hepatic_failure
	 * immunosuppression
	 * leukemia
	 * lymphoma
	 * solid_tumor_with_metastasis
	 * hospital_death
	 * ethnicity
	 * gender
	 * age
	 * bmi


In [139]:
df.head(10)

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0.0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0.0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,1

In [83]:
df[df.sodium_apache.isna()].hospital_death.value_counts()[1]/df[df.sodium_apache.isna()].hospital_death.value_counts()[0]

0.07950519774667518

In [74]:
df.shape

(91688, 186)

In [34]:
# d = dict.fromkeys(['columns', 'nulls', 'ratio', 'val1', 'val'])
list = []
for c in df.columns:
    list1 = []
    val = df[df[c].notna()].hospital_death.value_counts()[1]/df[df[c].notna()].hospital_death.value_counts()[0]
    if df[c].isna().sum() > 0:
        val1 = df[df[c].isna()].hospital_death.value_counts()[1]/df[df[c].isna()].hospital_death.value_counts()[0]
        if abs(val1-val) > 0.05:
            list1.append(c)
            list1.append(df[c].isna().sum())
            list1.append(val1/val)
            list1.append(val1)
            list1.append(val)
            list1.append(df[df[c].isna()].hospital_death.value_counts()[1])


            list.append(list1)

# print(list)
#             print(f'{c}: total_nulls: {df[c].isna().sum()} , ratio: {val1/val},  val1:{val1}, val:{val}')


In [35]:
df2 = pd.DataFrame.from_records(list)

In [36]:
df2.columns = ['column', 'num_nulls', 'val1_val_ratio', 'null_deathrate', 'not_null_deathrate', 'null_deaths']

val: Deathrate when value not null
val1: Deathrate when value is null
ratio: val1/val: Deathrate null/deathrate nontnull

In [39]:
df2.sort_values(by=['not_null_deathrate'], ascending = False)

,column,num_nulls,val1_val_ratio,null_deathrate,not_null_deathrate,null_deaths
45,h1_lactate_max,84369,0.386983,0.084797,0.219124,6595
46,h1_lactate_min,84369,0.386983,0.084797,0.219124,6595
8,paco2_apache,70868,0.290996,0.063047,0.216658,4203
11,ph_apache,70868,0.290996,0.063047,0.216658,4203
10,pao2_apache,70868,0.290996,0.063047,0.216658,4203
4,fio2_apache,70868,0.290996,0.063047,0.216658,4203
9,paco2_for_ph_apache,70868,0.290996,0.063047,0.216658,4203
54,d1_pao2fio2ratio_min,66008,0.266918,0.055655,0.208510,3480
53,d1_pao2fio2ratio_max,66008,0.266918,0.055655,0.208510,3480
37,d1_lactate_max,68396,0.289705,0.060370,0.208385,3894


In [183]:
df2['factor'] = df2.val1_val_ratio * df2.val1_deaths

In [191]:
df2 = df2[df2.val1_val_ratio > 1.5]

In [192]:
df2.sort_values(by=['factor'], ascending = False)

,column,num_nulls,val1_val_ratio,val1,val,val1_deaths,factor
10,temp_apache,4103,1.718537,0.157405,0.091592,558,958.943855
3,gcs_eyes_apache,1898,2.257496,0.208148,0.092203,327,738.201227
4,gcs_motor_apache,1898,2.257496,0.208148,0.092203,327,738.201227
5,gcs_verbal_apache,1898,2.257496,0.208148,0.092203,327,738.201227
28,d1_temp_min,2320,1.705191,0.158263,0.092812,317,540.545535
27,d1_temp_max,2320,1.705191,0.158263,0.092812,317,540.545535
0,height,1328,1.743245,0.162872,0.093430,186,324.243505
21,d1_spo2_max,331,1.891008,0.177936,0.094096,50,94.550409
22,d1_spo2_min,331,1.891008,0.177936,0.094096,50,94.550409
25,d1_sysbp_max,158,2.592621,0.244094,0.094150,31,80.371240


In [196]:
df[df['temp_apache'].isna()].hospital_death.value_counts()

0    3545
1     558
Name: hospital_death, dtype: int64

In [198]:
df[df['temp_apache'].notna()].hospital_death.value_counts()

0    80236
1     7349
Name: hospital_death, dtype: int64

In [189]:
=

0.2440944881889764

In [177]:
df[df['paco2_for_ph_apache'].notna()].hospital_death.value_counts()

0    17132
1     3708
Name: hospital_death, dtype: int64

In [213]:
df[df.gcs <4].hospital_death.value_counts()

0    3251
1    1819
Name: hospital_death, dtype: int64

In [214]:
df[df.gcs >= 4].hospital_death.value_counts()

0    78959
1     5761
Name: hospital_death, dtype: int64

In [274]:
df[(df.intubated_apache==1) & (df.gcs <8) & (df.ventilated_apache == 1) & (df.d1_pao2fio2ratio_min <100) ].hospital_death.value_counts()


1    505
0    467
Name: hospital_death, dtype: int64

In [233]:
df

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,gcs
0,66154,25312,118,0,68.0,22.730000,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0.0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0.0,14.10,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.10,14.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,13.0
1,114252,59342,81,0,77.0,27.420000,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0

In [253]:
df['diag'] = df['apache_3j_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]

In [273]:
pd.Series(df.groupby('diag').hospital_death.mean()).sort_values()

diag
902     0.000000
2101    0.000000
1803    0.000000
903     0.000000
1704    0.000000
          ...   
312     0.213483
1501    0.256881
202     0.279188
101     0.290837
102     0.498003
Name: hospital_death, Length: 113, dtype: float64

In [271]:
(df['diag']==903).sum()

0

In [276]:
df.heart_rate_apache.min()

30.0

In [10]:
to_remove=['hospital_id','icu_id','patient_id','encounter_id','apache_4a_hospital_death_prob','apache_4a_icu_death_prob' ]

features = [col for col in df.columns if col not in to_remove]

In [26]:
df1 = pd.read_csv('data/WiDS Datathon 2020 Dictionary.csv')
df1 = df1[(df1['Data Type'] == 'string') | (df1['Data Type'] == 'binary')]
cat_features = list(df1['Variable Name'].values)
for c in cat_features:
    if c not in df1.columns or c == 'hospital_death':
        cat_features.remove(c)

num_feature = [col for col in features if col not in cat_features and df[col].dtype != 'object']
drop_columns=[]
corr = df[num_feature].corr()


# Drop highly correlated features 
columns = np.full((corr.shape[0],), True, dtype=bool)

for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >=0.99 :
            if columns[j]:
                columns[j] = False
                print('col1: {} col2: {} - Correlation: {}'.format(df[num_feature].columns[i] , df[num_feature].columns[j], corr.iloc[i,j]))
        elif corr.iloc[i,j] <= -0.99:
            if columns[j]:
                columns[j] = False

drop_columns = df[num_feature].columns[columns == False].values
print(drop_columns)

col1: bilirubin_apache col2: d1_bilirubin_max - Correlation: 0.996567653837065
col1: bun_apache col2: d1_bun_max - Correlation: 0.9911213087008959
col1: creatinine_apache col2: d1_creatinine_max - Correlation: 0.9941462301078292
col1: paco2_apache col2: paco2_for_ph_apache - Correlation: 1.0
col1: d1_diasbp_max col2: d1_diasbp_noninvasive_max - Correlation: 0.9978521575920084
col1: d1_diasbp_min col2: d1_diasbp_noninvasive_min - Correlation: 0.9968783973915579
col1: d1_mbp_min col2: d1_mbp_noninvasive_min - Correlation: 0.9952283145634516
col1: d1_sysbp_max col2: d1_sysbp_noninvasive_max - Correlation: 0.9965574914047141
col1: d1_sysbp_min col2: d1_sysbp_noninvasive_min - Correlation: 0.9963192438490275
col1: h1_diasbp_max col2: h1_diasbp_noninvasive_max - Correlation: 0.9919818943587326
col1: h1_mbp_max col2: h1_mbp_noninvasive_max - Correlation: 0.9931158503969355
col1: h1_mbp_min col2: h1_mbp_noninvasive_min - Correlation: 0.9933153849044118
col1: h1_sysbp_max col2: h1_sysbp_noninva

In [12]:
corr = df[num_feature].corr()

In [19]:
columns = np.full((corr.shape[0]), True, dtype=bool)

In [21]:
corr = df[num_feature].corr()

In [22]:
corr

,age,elective_surgery,height,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_3j_diagnosis,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,diabetes_mellitus,immunosuppression,lymphoma
age,1.000000,0.063073,-0.112053,0.047862,NaN,-0.131678,-0.116707,-0.056808,-0.003308,-0.046046,0.235321,0.055652,0.037489,0.026430,0.025795,-0.017996,0.005216,-0.147434,-0.118238,0.012057,-0.016233,0.032183,0.032183,-0.019847,0.045634,0.038847,0.038165,-0.082249,-0.177521,-0.010400,-0.188844,-0.249668,-0.061383,-0.210160,-0.058953,-0.207958,-0.133940,-0.141757,-0.027033,-0.112405,0.006314,-0.130570,0.006367,-0.129083,0.032312,0.036250,-0.037770,-0.083361,0.031536,0.009404,0.105872,-0.059204,0.105273,-0.059425,-0.084331,-0.068930,-0.245784,-0.268235,-0.145066,-0.197940,-0.135949,-0.190558,-0.163022,-0.162612,-0.051722,-0.119551,-0.058482,-0.107211,-0.056827,-0.102458,0.027862,0.033907,-0.059819,-0.072688,0.038730,0.000272,0.047217,-0.004705,0.046716,-0.002982,-0.090826,-0.085390,-0.128494,-0.104572,-0.045753,-0.042298,0.236325,0.263972,0.023828,0.046478,0.052248,0.070536,0.010252,0.065457,0.063841,0.097099,-0.154622,-0.142785,-0.122869,-0.119204,0.105560,0.116207,0.021097,0.042710,-0.081830,-0.081064,0.059642,0.110208,0.008520,0.038206,-0.003166,-0.001111,-0.119258,-0.119307,-0.034530,-0.034426,0.175648,0.176090,0.071826,0.070612,0.024254,0.025124,-0.075144,-0.060239,0.165599,0.165679,-0.143409,-0.139434,-0.119478,-0.122617,0.105560,0.116207,-0.006292,-0.004193,-0.102245,-0.102599,0.040184,0.039869,0.028442,0.025688,-0.052842,-0.051901,0.03